In [1]:
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-mnwdm5d_
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-mnwdm5d_
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
import matplotlib.pyplot as plt # plotting
import matplotlib.image as mpimg
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json
import torch
from collections import Counter
from PIL import Image
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import clip
#from google.colab import drive
import os
from os import listdir
from os.path import isfile, join
#drive.mount("/content/gdrive")
#os.chdir("dataset/train/train")
#os.chdir("dataset/test/test")
#os.chdir("dataset/val/val")

In [3]:
def find_most_common_answer(answers):
    answer_counter = Counter(answers)
    most_common_answers = answer_counter.most_common()
    most_common_answer, count = most_common_answers[0]
    return most_common_answer

def select_most_common_answers(df):
    selected_answers = []
    for idx, row in df.iterrows():
        answers = [answer["answer"] for answer in row["answers"]]
        selected_answer = find_most_common_answer(answers)

        selected_answers.append({"answer": selected_answer
        })

    # Update the "answer" and "answer_confidence" columns in the DataFrame
    df[["answer"]] = pd.DataFrame(selected_answers)

    return df.drop(["answers"], axis=1)

def plot_loss(train_loss, val_loss):
    epochs = range(1, len(train_loss) + 1)

    plt.plot(epochs, train_loss, label='Training Loss')
    plt.plot(epochs, val_loss, label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.show()

def plot_img(path):
    # Load the JPEG image
    image = mpimg.imread(path)

    # Plot the image
    plt.imshow(image)
    plt.axis('off')  # Remove axis ticks and labels
    plt.show()

In [4]:
def dataloader_json(path,test=False):
    # Load the JSON file
    with open(path, 'r') as f:
        data = json.load(f)
    # Create a DataFrame from the loaded JSON data
    df = pd.DataFrame(data)

    if test:
        return df

    return select_most_common_answers(df)

In [5]:
train_df = dataloader_json("dataset/Annotations/Annotations/new_train.json")

train_df

,image,question,answer_type,answerable,answer
0,VizWiz_train_00000000.jpg,What's the name of this product?,other,1,basil leaves
1,VizWiz_train_00000001.jpg,Can you tell me what is in this can please?,other,1,coca cola
2,VizWiz_train_00000002.jpg,Is this enchilada sauce or is this tomatoes? ...,other,1,tomatoes
3,VizWiz_train_00000003.jpg,What is the captcha on this screenshot?,other,1,t36m
4,VizWiz_train_00000004.jpg,What is this item?,other,1,solar garden light
...,...,...,...,...,...
20638,VizWiz_train_00002836.jpg,What is the length of the tortoise's necks?,other,1,variation
20639,VizWiz_train_00002836.jpg,What is the date of the last day of the week?,other,1,date
20640,VizWiz_train_00002836.jpg,What is the nature of the species?,other,1,observations of many species and their geograp...
20641,VizWiz_train_00002836.jpg,What is the name of the systme that is used to...,other,1,systme


In [6]:



train_df['answer'].value_counts()

answer
unanswerable                      3941
unsuitable                        3038
no                                 567
yes                                516
white                              298
                                  ... 
rugrats                              1
coffee cup eiffel tower statue       1
after shave                          1
light bulb white                     1
basil leaves                         1
Name: count, Length: 5660, dtype: int64

In [7]:
val_df = dataloader_json("dataset/Annotations/Annotations/val.json")
val_df

,image,question,answer_type,answerable,answer
0,VizWiz_val_00000000.jpg,Ok. There is another picture I hope it is a be...,unanswerable,0,unanswerable
1,VizWiz_val_00000001.jpg,Can you tell me what this medicine is please?,other,1,night time
2,VizWiz_val_00000002.jpg,What is the title of this book?,other,1,dog years
3,VizWiz_val_00000003.jpg,Which one is the blue one?,other,1,right
4,VizWiz_val_00000004.jpg,What does the arrow say?,unanswerable,0,unanswerable
...,...,...,...,...,...
4314,VizWiz_val_00004314.jpg,what is this?,other,1,salad dressing
4315,VizWiz_val_00004315.jpg,Is this modern?,unanswerable,0,unsuitable
4316,VizWiz_val_00004316.jpg,I need to buy this battery for my cordless pho...,unanswerable,0,unanswerable
4317,VizWiz_val_00004317.jpg,What kind of mix is this?,other,1,cake


In [8]:
val_df['answer'].nunique()

1531

In [9]:
data_df = pd.concat((train_df,val_df), axis =0,ignore_index=True)
data_df

,image,question,answer_type,answerable,answer
0,VizWiz_train_00000000.jpg,What's the name of this product?,other,1,basil leaves
1,VizWiz_train_00000001.jpg,Can you tell me what is in this can please?,other,1,coca cola
2,VizWiz_train_00000002.jpg,Is this enchilada sauce or is this tomatoes? ...,other,1,tomatoes
3,VizWiz_train_00000003.jpg,What is the captcha on this screenshot?,other,1,t36m
4,VizWiz_train_00000004.jpg,What is this item?,other,1,solar garden light
...,...,...,...,...,...
24957,VizWiz_val_00004314.jpg,what is this?,other,1,salad dressing
24958,VizWiz_val_00004315.jpg,Is this modern?,unanswerable,0,unsuitable
24959,VizWiz_val_00004316.jpg,I need to buy this battery for my cordless pho...,unanswerable,0,unanswerable
24960,VizWiz_val_00004317.jpg,What kind of mix is this?,other,1,cake


In [10]:
ans_lb = LabelEncoder()
data_df['answer'] = ans_lb.fit_transform(data_df['answer'])
ans_type_lb = LabelEncoder()
data_df['answer_type']= ans_type_lb.fit_transform(data_df['answer_type'])

In [11]:
data_df

,image,question,answer_type,answerable,answer
0,VizWiz_train_00000000.jpg,What's the name of this product?,1,1,779
1,VizWiz_train_00000001.jpg,Can you tell me what is in this can please?,1,1,1739
2,VizWiz_train_00000002.jpg,Is this enchilada sauce or is this tomatoes? ...,1,1,5911
3,VizWiz_train_00000003.jpg,What is the captcha on this screenshot?,1,1,5720
4,VizWiz_train_00000004.jpg,What is this item?,1,1,5403
...,...,...,...,...,...
24957,VizWiz_val_00004314.jpg,what is this?,1,1,5082
24958,VizWiz_val_00004315.jpg,Is this modern?,2,0,6068
24959,VizWiz_val_00004316.jpg,I need to buy this battery for my cordless pho...,2,0,6059
24960,VizWiz_val_00004317.jpg,What kind of mix is this?,1,1,1281


In [12]:
val = data_df.iloc[20523:].reset_index(drop=True)
val

,image,question,answer_type,answerable,answer
0,VizWiz_train_00000006.jpg,What is the name of the plant that is a Citrus?,1,1,44
1,VizWiz_train_00000006.jpg,What is the amount of tartar that is built up?,1,1,43
2,VizWiz_train_00000006.jpg,What is the name of the plant that grows on th...,1,1,476
3,VizWiz_train_00000006.jpg,Is brushing alone better than brushing with a ...,1,1,878
4,VizWiz_train_00000006.jpg,What is the name of the substance that is used...,1,1,3629
...,...,...,...,...,...
4434,VizWiz_val_00004314.jpg,what is this?,1,1,5082
4435,VizWiz_val_00004315.jpg,Is this modern?,2,0,6068
4436,VizWiz_val_00004316.jpg,I need to buy this battery for my cordless pho...,2,0,6059
4437,VizWiz_val_00004317.jpg,What kind of mix is this?,1,1,1281


In [13]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model_clip, preprocess = clip.load("ViT-L/14", device=device)

print(f'Using {device}')

Using cuda


In [14]:
encodings = []
for img , question in tqdm(zip(train_df['image'],train_df['question'])):
    if "train" in img:

        image = preprocess(Image.open(f'dataset/train/train/{img}').rotate(90)).unsqueeze(0).to(device)
    elif "test" in img:
       image = preprocess(Image.open(f'dataset/test/test/{img}')).unsqueeze(0).to(device)
    else:
        image = preprocess(Image.open(f'dataset/val/val/{img}')).unsqueeze(0).to(device)

    text = clip.tokenize(question).to(device)

    with torch.no_grad():
        image_encoding = model_clip.encode_image(image)
        text_encoding = model_clip.encode_text(text)

    encodings.append(torch.cat([image_encoding, text_encoding], dim=-1))


20643it [24:28, 14.05it/s]


In [15]:
torch.save(encodings,"encd768-90.pt")


In [16]:
# Generate the indices for train-test split
indices = np.arange(20523)

# Perform train-test split
train_indices, test_indices = train_test_split(indices, test_size=0.05, random_state=42, stratify=data_df.iloc[:20523]['answer_type'])

In [17]:
train_indices_aug = train_indices + 20643

In [18]:
train = data_df.iloc[train_indices] 
train = train.reset_index(drop=True)
train

,image,question,answer_type,answerable,answer
0,VizWiz_train_00014709.jpg,What is this?,1,1,5949
1,VizWiz_train_00009566.jpg,What product is this? Is it moisturizing lotion?,1,1,6482
2,VizWiz_train_00011322.jpg,What's in this box?,2,0,6059
3,VizWiz_train_00005267.jpg,What is in this kitchen? In this room?,1,1,2551
4,VizWiz_train_00009281.jpg,Is this shampoo or conditioner?,1,1,5210
...,...,...,...,...,...
19491,VizWiz_train_00015453.jpg,What number is shown?,0,1,238
19492,VizWiz_train_00019967.jpg,Where is B?,1,1,3679
19493,VizWiz_train_00017226.jpg,What kind of tea is this?,1,1,4299
19494,VizWiz_train_00016495.jpg,What is this?,1,1,1662


In [19]:
test = data_df.iloc[test_indices]
test = test.reset_index(drop=True)
test

,image,question,answer_type,answerable,answer
0,VizWiz_train_00023579.jpg,Sometimes they're able to and sometimes they a...,2,0,6059
1,VizWiz_train_00001384.jpg,What is the title of this book? Thank you,2,0,6059
2,VizWiz_train_00015850.jpg,What color is this?,1,1,4589
3,VizWiz_train_00010122.jpg,What's being displayed on my screen?,1,1,407
4,VizWiz_train_00016891.jpg,What is this item?,1,1,2921
...,...,...,...,...,...
1022,VizWiz_train_00010922.jpg,What flavor of kool aid is this?,1,1,6003
1023,VizWiz_train_00017572.jpg,What is this item?,1,1,4915
1024,VizWiz_train_00016464.jpg,What is the name of this book? Thank you.,1,1,3660
1025,VizWiz_train_00001537.jpg,What is this?,1,1,6391


In [20]:




embedding_size = 768
classes = len(np.unique(ans_lb.classes_))
aux_classes = len(np.unique(train['answer_type']))
BATCH_SIZE = 64

In [21]:
classes, aux_classes

(6532, 4)

In [22]:
class dataset(Dataset):
    def __init__(self, indices, answers, types, length):
        self.indices = indices
        self.answers = answers
        self.types = types
        self.length = length

    def __getitem__(self, index):
            if self.length <= 20523:
                return encd[self.indices[index]].float() , torch.tensor(int(self.answers[index])), torch.tensor(int(self.types[index]))
            
            return encd[self.indices[index]].float() , torch.tensor(int(self.answers[index % (self.length/2)])), torch.tensor(int(self.types[index % (self.length/2)]))
            
    def __len__(self):
          return self.length


In [23]:
encd = torch.cat([torch.load('dataset/encALL.pt')])

In [24]:
class dataset(Dataset):
    def __init__(self, indices, answers, types, length):
        self.indices = indices
        self.answers = answers
        self.types = types
        self.length = length

    def __getitem__(self, index):
            if self.length <= 20523:
                return encd[self.indices[index]].float() , torch.tensor(int(self.answers[index])), torch.tensor(int(self.types[index]))
            
            return encd[self.indices[index]].float() , torch.tensor(int(self.answers[index % (self.length/2)])), torch.tensor(int(self.types[index % (self.length/2)]))
            
    def __len__(self):
          return self.length

In [25]:
trainDataset = dataset(np.concatenate((train_indices, train_indices_aug)), train['answer'], train['answer_type'], len(train_indices)*2)


valDataset = dataset(np.array(list(range(encd.shape[0] - 4319, encd.shape[0]))), val['answer'], val['answer_type'], 4319)

train_dataloader = DataLoader(trainDataset, batch_size = BATCH_SIZE, shuffle = True, num_workers=0)

val_dataloader = DataLoader(valDataset, batch_size = BATCH_SIZE, shuffle = True, num_workers=0)

In [26]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.ln1 = nn.LayerNorm(embedding_size*2)
        self.dp1 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(embedding_size* 2, 512)
        
        self.ln2 = torch.nn.LayerNorm(512)
        self.dp2 = torch.nn.Dropout(0.5)
        
        self.fc2 = torch.nn.Linear(512, classes)
        
        self.fc_aux = torch.nn.Linear(512, aux_classes)
        
        self.lnaux = torch.nn.LayerNorm(aux_classes)
        self.dpaux = torch.nn.Dropout(0.5)
                
        self.fc_gate = torch.nn.Linear(aux_classes, classes)
        self.act_gate = torch.nn.Sigmoid()

    def forward(self, x):
        x = self.ln1(x)
        x = self.dp1(x)
        
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        
        x = self.ln2(x)
        x = self.dp2(x)
        
#         aux
        aux = self.fc_aux(x)
#         aux = self.lnaux(aux)
#         aux = self.dpaux(aux)
        
#         linear bottom
        vqa = self.fc2(x)
        
        output = vqa * self.act_gate(self.fc_gate(aux))
        
        
        return output,aux

In [27]:
def run_model(model,dataloader,val_loader, optimizer,train = True ):
    if train:
        model.train()
    
    loss = nn.CrossEntropyLoss()
    
    total_loss = 0
    total_correct = 0
    total_samples = 0
    
    total_correct_ty = 0
    total_samples_ty = 0
    
    for (data, ans , ans_type) in tqdm(dataloader):
        data = data.to(device)
        ans = ans.to(device)
        ans_type = ans_type.to(device)
        optimizer.zero_grad()
        output , aux = model(data)
        
        loss_ans = loss(output, ans)
        loss_type = loss(aux,ans_type)
        loss_combined=loss_ans+loss_type
        total_loss += loss_combined.item()
        loss_combined.backward()
        optimizer.step()
        
        
#         Answer Accurracy
        _, predicted_labels = torch.max(output, dim=1)
        correct = (predicted_labels == ans).sum().item()
        total_correct += correct
        total_samples += ans.size(0)
        train_accuracy = total_correct / total_samples
        
#         Type Accuracy
        _, predicted_labels_ty = torch.max(aux, dim=1)
        correct_ty = (predicted_labels_ty == ans_type).sum().item()
        total_correct_ty += correct_ty
        total_samples_ty += ans_type.size(0)
        train_accuracy_ty = total_correct_ty / total_samples_ty
    
    total_train_accuracy = (train_accuracy + train_accuracy_ty) / 2
        
        
    if val_loader is not None:
        model.eval()

    # Initialize validation-specific variables
    val_loss = 0.0
    total_correct_val=0
    total_samples_val=0
    
    total_correct_val_ty=0
    total_samples_val_ty=0
    
    # Disable gradient calculation
    with torch.no_grad():
        # Iterate over the validation data
        for (data, ans , ans_type) in val_loader:
            data = data.to(device)
            ans = ans.to(device)
            ans_type = ans_type.to(device)
            
            # Forward pass
            output , aux = model(data)

            # Compute the loss
            loss_ans = loss(output, ans)
            loss_type = loss(aux,ans_type)
            loss_combined=loss_ans+loss_type
            val_loss += loss_combined.item()

            # Update validation metrics
            _, val_predicted = torch.max(output, dim=1)
            correct_val = (val_predicted == ans).sum().item()
            total_correct_val += correct_val
            total_samples_val += ans.size(0)
            val_accuracy = total_correct_val / total_samples_val
            
            _, val_predicted_ty = torch.max(aux, dim=1)
            correct_val_ty = (val_predicted_ty == ans_type).sum().item()
            total_correct_val_ty += correct_val_ty
            total_samples_val_ty += ans_type.size(0)
            val_accuracy_ty = total_correct_val_ty / total_samples_val_ty
        
        total_val_accuracy = (val_accuracy + val_accuracy_ty) / 2
        
    train_loss = total_loss/len(dataloader)
    val_loss = val_loss/len(val_loader)
    
    print(f"\nTrain Loss: {train_loss:.4f} | AVG Train ACC: {total_train_accuracy * 100:.4f}% | Val Loss: {val_loss:.4f} | AVG Val ACC: {total_val_accuracy * 100:.2f}%")
    
    print(f"\nTrain ANS ACC: {train_accuracy * 100:.4f}% | VAL ANS ACC: {val_accuracy * 100:.4f}% | Train TYPE ACC: {train_accuracy_ty * 100:.4f}% | VAL TYPE ACC: {val_accuracy_ty * 100:.2f}%\n")
        
    return train_loss, val_loss


In [28]:

device = "cuda" if torch.cuda.is_available() else "cpu"
model = Model()
model = model.to(device)
model = nn.DataParallel(model)



In [34]:
import sys
from importlib import reload
if sys.version[0] == '2':
    reload(sys)
    sys.setdefaultencoding("utf-8")
print('Hi')

Hi


In [35]:
testDataset = dataset(test_indices,test['answer'], test['answer_type'],1027)
test_dataloader = DataLoader(testDataset, batch_size=BATCH_SIZE , shuffle=True, num_workers=0)

In [36]:
print(test_dataloader)

In [37]:
loss = nn.CrossEntropyLoss()
model.eval()
test_loss = 0.0
total_correct_test=0
total_samples_test=0
    
total_correct_test_ty=0
total_samples_test_ty=0

# Disable gradient calculation
with torch.no_grad():
    # Iterate over the validation data
    for (data, ans , ans_type) in test_dataloader:
            data = data.to(device)
            ans = ans.to(device)
            ans_type = ans_type.to(device)
            # Forward pass
            output , aux = model(data)

            # Compute the loss
            loss_ans = loss(output, ans)
            loss_type = loss(aux,ans_type)
            loss_combined=loss_ans+loss_type
            test_loss += loss_combined.item()

            # Update validation metrics
            _, test_predicted = torch.max(output, dim=1)
            correct_test = (test_predicted == ans).sum().item()
            total_correct_test += correct_test
            total_samples_test += ans.size(0)
            test_accuracy = total_correct_test / total_samples_test
            
            _, test_predicted_ty = torch.max(aux, dim=1)
            correct_test_ty = (test_predicted_ty == ans_type).sum().item()
            total_correct_test_ty += correct_test_ty
            total_samples_test_ty += ans_type.size(0)
            test_accuracy_ty = total_correct_test_ty / total_samples_test_ty
        
    total_test_accuracy = (test_accuracy + test_accuracy_ty) / 2
        
        
print(f"Test TYPE ACC: {test_accuracy_ty * 100:.4f}% | Test ANS ACC: {test_accuracy * 100:.4f}% | AVG Test ACC: {total_test_accuracy * 100:.4f}% | Test Loss: {test_loss:.4f}") 






Test TYPE ACC: 24.2454% | Test ANS ACC: 0.0000% | AVG Test ACC: 12.1227% | Test Loss: 174.0864


In [ ]:
model.eval();